# 재정정보 AI 검색 알고리즘
Answering with Retrieval
- 중앙정부 재정 정보에 대한 검색 기능을 개선, 활용도를 높이는 질의응답 알고리즘을 개발
- 평가 방법: 문자 단위의 F1 score


# RAG & LangChain이란
## RAG(Retrieval-Augmented Generation) 이란
- RAG는 외부 정보(문서 등)을 검색해서 LLM의 응답 생성을 도와주는 구조
- LLM은 훈련기점 이후의 최신 정보를 모른다.
- 세부적인 도메인 지식에 대해 모호하게 답변할 수 있으므로
- 검색(Retrieval)을 먼저 수행한 뒤 생성(Generation)하는 구조
- https://research.ibm.com/blog/retrieval-augmented-generation-RAG
- 사용자가 query input -> query를 vector로 embedding -> vector db에서 유사 문서 검색() -> top-k 문서 반환 -> 프롬프트 생성(문서 + query를 하나의 프롬프트로 조립) -> llm 응답 생성

| 구성요소                   | 설명                                                                         |
| ---------------------- | -------------------------------------------------------------------------- |
| **1. Retriever**       | 질문과 관련된 문서를 외부 지식 저장소(벡터 DB 등)에서 검색함. 대표적으로 FAISS, Chroma, Weaviate 사용     |
| **2. Vector Store**    | 문서들을 미리 임베딩 벡터로 변환해 저장해 둔 벡터 데이터베이스                                        |
| **3. Embedding Model** | 문서 및 질문을 벡터로 변환하기 위한 모델. 예: `sentence-transformers`, `OpenAI Embeddings` 등 |
| **4. Prompt Template** | 검색된 문서들을 묶고, 질문과 함께 LLM에 전달할 프롬프트 구성                                       |
| **5. Generator (LLM)** | 위 프롬프트를 기반으로 답변 생성. 예: GPT-4, LLaMA, Claude 등                              |


## LangChain이란
- LLM 기반으로 복잡한 애플리케이션을 쉽게 개발할 수 있도록 도와주는 프레임 워크

# Baseline
- 평가 데이터셋만 활용
- source pdf 마다 vector db 구축
- langchain 라이브러리, llama-2-ko-7b 모델을 사용하여 RAG 프로세스를 통해 추론
- test_set에 대한 추론만 진행

## Install & Import

In [ ]:
! pip install accelerate
!pip install -i https://pypi.org/simple/bitsandbytes
!pip install transformers[torch] -U
!pip install datasets
!pip install langchain
!pip install langchain_community
!pip install PyMuPDF
!pip install sentence-transformers

In [ ]:
!pip install faiss-cpu

In [ ]:
!pip install bitsandbytes

In [ ]:
!pip install -U bitsandbytes

In [6]:
import os
import unicodedata   # 유니코드 문자 정규화 및 분석
import torch  # 딥러닝 프레임워크
import pandas as pd
from tqdm import tqdm   # 학습, 평가 시 작업 진행률 표시 바
import fitz  #PyMuPDF : PDF 읽기/ 페이지 추출 기능

from transformers import (
    AutoTokenizer,   # pre-trained model에 맞는 tokenizer 자동 로드
    AutoModelForCausalLM,   #텍스트 생성용 LLM model 자동 로드
    pipeline, # 텍스트 생성, 분류, 요약, 추론 을 쉽게할 수 있는 api
    BitsAndBytesConfig   # 모델 압축
)
from accelerate import Accelerator   # 분산학습, 혼합 정밀도 학습 등을 쉽게 설명해주는 huggingface 도구( gpu 자동 분배 등)



#langchain 관련
from langchain.llms import huggingface_pipeline
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

| 역할            | 사용하는 라이브러리                                                 | 설명                 |
| ------------- | ---------------------------------------------------------- | ------------------ |
| 문서 읽기 (PDF 등) | `fitz`                                                     | 문서 내 텍스트 추출        |
| 텍스트 분할        | `RecursiveCharacterTextSplitter`                           | 벡터화하기 전 문서 조각 분할   |
| 임베딩           | `HuggingFaceEmbeddings`                                    | 각 문서 조각을 벡터로 변환    |
| 벡터 검색         | `FAISS`                                                    | 임베딩된 문서를 빠르게 검색    |
| LLM 설정        | `huggingface_pipeline`, `AutoModelForCausalLM`, `pipeline` | LLM을 래핑하여 텍스트 생성   |
| 프롬프트 설정       | `PromptTemplate`                                           | LLM에 넣을 질의 구조 설정   |
| 파이프라인 조립      | `RunnablePassthrough`, `StrOutputParser`                   | 입력 → 처리 → 출력 흐름 구성 |
| 학습/추론 최적화     | `Accelerator`, `BitsAndBytesConfig`                        | 학습 속도 및 메모리 최적화    |


## Vector DB

In [10]:
def process_pdf(file_path, chunk_size = 800, chunk_overlap = 50):
  '''pdf 텍스트 추출 후 chunk 단위로 나누는 함수 '''
  #pdf 파일 열기
  doc = fitz.open(file_path)
  text = ''

  #모든 페이지의 텍스트 추출
  for page in doc:
    text += page.get_text()

  #텍스트를 chunk로 분할
  splitter = RecursiveCharacterTextSplitter(
      chunk_size = chunk_size,
      chunk_overlap=chunk_overlap
  )
  chunk_temp = splitter.split_text(text)

  #Document 객체 리스트 생성
  chunks = [Document(page_content = t) for t in chunk_temp]

  return chunks

In [11]:
def create_vector_db(chunks, model_path = 'intfloat/multilingual-e5-small'):
  '''FAISS DB 생성'''

  #임베딩 모델 설정
  model_kwargs = {'device':'cuda'}
  encode_kwargs = {'normalize_embeddings' : True}
  embeddings = HuggingFaceEmbeddings(
      model_name = model_path,
      model_kwargs = model_kwargs,
      encode_kwargs = encode_kwargs
  )


  #FAISS DB 생성 및 변환
  db = FAISS.from_documents(chunks, embedding = embeddings)

  return db


In [12]:
def normalize_path(path):
  '''경로 유니코드 정규화'''
  return unicodedata.normalize('NFC', path)

In [13]:
def process_pdfs_from_dataframe(df, base_directory):
  '''딕셔너리에 pdf명을 키로하여 db retriever 저장'''
  pdf_databases = {}
  unique_paths = df['Source_path'].unique()

  for path in tqdm(unique_paths, desc = 'Processing PDFs'):
    #경로 정규화 및 절대 경로 생성
    normalized_path = normalize_path(path)
    full_path = os.path.normpath(os.path.join(base_directory, normalized_path.lstrip('./'))) if not os.path.isabs(normalized_path) else normalized_path

    pdf_title = os.path.splitext(os.path.basename(full_path))[0]
    print(f'Processing{pdf_title}...')

    #PDF 처리 및 벡터 DB 생성
    chunks = process_pdf(full_path)
    db = create_vector_db(chunks)

    #Retriever 생성
    retriever = db.as_retriever(search_type = 'mmr',
                                search_kwargs={'k':3, 'fetch_k':8})

    #결과 저장
    pdf_databases[pdf_title] = {
        'db': db,
        'retriever':retriever
    }

  return pdf_databases

## DB 생성

In [8]:
# 깃허브 저장소 압축 다운로드
!wget https://github.com/s-jyeon/dacon_playgroud/archive/refs/heads/main.zip -O repo.zip

# 압축 해제
!unzip -q repo.zip

# 작업 디렉토리 이동
%cd dacon_playgroud-main/NLP_practice/재정정보AI검색알고리즘

--2025-06-08 13:06:37--  https://github.com/s-jyeon/dacon_playgroud/archive/refs/heads/main.zip
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/s-jyeon/dacon_playgroud/zip/refs/heads/main [following]
--2025-06-08 13:06:38--  https://codeload.github.com/s-jyeon/dacon_playgroud/zip/refs/heads/main
Resolving codeload.github.com (codeload.github.com)... 140.82.113.9
Connecting to codeload.github.com (codeload.github.com)|140.82.113.9|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘repo.zip’

repo.zip                [       <=>          ]  56.12M  21.2MB/s    in 2.6s    

2025-06-08 13:06:40 (21.2 MB/s) - ‘repo.zip’ saved [58845318]

/content/dacon_playgroud-main/NLP_practice/재정정보AI검색알고리즘


In [7]:
base_directory = './data'
df = pd.read_csv('./data/test.csv')
pdf_databases = process_pdfs_from_dataframe(df, base_directory)

Processing PDFs:   0%|          | 0/9 [00:00<?, ?it/s]<ipython-input-3-7b106b319cd2>:7: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


Processing중소벤처기업부_혁신창업사업화자금(융자)...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Processing PDFs:  11%|█         | 1/9 [00:09<01:15,  9.46s/it]

Processing보건복지부_부모급여(영아수당) 지원...


Processing PDFs:  22%|██▏       | 2/9 [00:14<00:47,  6.86s/it]

...

Processing PDFs:  78%|███████▊  | 7/9 [00:43<00:12,  6.09s/it]

Processing「FIS 이슈&포커스」 22-2호 《재정성과관리제도》...


Processing PDFs:  89%|████████▉ | 8/9 [00:49<00:05,  5.94s/it]

Processing「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》...


Processing PDFs: 100%|██████████| 9/9 [00:54<00:00,  6.03s/it]


## MODEL Import

In [8]:
from langchain.llms import HuggingFacePipeline
def setup_llm_pipeline():
  #4비트 양자화 설정 : 딥러닝 모델의 파라미터를 4bit 표현으로 바꾸어 모델크기, 메모리 사용량을 줄임
  bnb_config = BitsAndBytesConfig(
      load_in_4bit = True,
      bnb_4bit_use_double_quant = True,
      bnb_4bit_quant_type='nf4',
      bnb_4bit_compute_dtype = torch.bfloat16
  )

  #모델 ID
  model_id = 'beomi/llama-2-ko-7b'

  #토크나이저 로드 및 설정
  tokenizer = AutoTokenizer.from_pretrained(model_id)
  tokenizer.use_default_system_prompt = False


  #모델 로드 및 양자화 설정 적용
  model = AutoModelForCausalLM.from_pretrained(
      model_id,
      quantization_config = bnb_config,
      device_map = 'auto',
      trust_remote_code = True
  )

  #HuggingFacePipeline 객체 생성
  text_generation_pipeline = pipeline(
      model = model,
      tokenizer = tokenizer,
      task = 'text-generation',
      temperature = 0.2,
      return_full_text = False,
      max_new_tokens = 128,
  )

  hf = HuggingFacePipeline(pipeline=text_generation_pipeline)

  return hf


#llm 파이프 라인
llm = setup_llm_pipeline()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

Device set to use cuda:0
<ipython-input-8-503fc83b84d2>:37: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  hf = HuggingFacePipeline(pipeline=text_generation_pipeline)


## Langchain을 이용한 추론

In [32]:
def normalize_string(s):
  '''유니코드 정규화'''
  return unicodedata.normalize('NFC', s)

In [33]:
def format_docs(docs):
  '''검색된 문서들을 하나의 문자열로 포맷팅'''
  context = ''
  for doc in docs:
    context += doc.page_content
    context += '\n'
  return context


In [ ]:
#결과를 저장할 리스트 초기화
results = []

#DataFrame의 각 행에 대해 처리
for _, row in tqdm(df.iterrows(), total = len(df), desc = 'Answering Questions'):

  #소스 문자열 정규화
  source = normalize_string(row['Source'])
  question = row['Question']


  #정규화된 키로 데이터베이스 검색
  normalized_keys = {normalize_string(k): v for k, v in pdf_databases.items()}
  retriever = normalized_keys[source]['retriever']

  #RAG 체인 구성
  template = """
  다음 정보를 바탕으로 질문에 답하세요:
  {context}

  질문 : {question}
  답변 :
  """
  prompt = PromptTemplate.from_template(template)

  #RAG 체인 정의
  rag_chain = (
      {'context': retriever | format_docs, 'question': RunnablePassthrough()}
      | prompt
      | llm
      | StrOutputParser()

  )

  #답변 추론
  print(f'Question : {question}')
  full_response = rag_chain.invoke(question)
  print(f'Answer: {full_response}\n')

  #결과 저장
  results.append({
      'Source':row['Source'],
      'Source_path': row['Source_path'],
      'Question': question,
      'Answer':full_response
  })

## Submission

In [19]:
#제출용 샘플 파일 로드
submit_df = pd.read_csv('./data/sample_submission.csv')

#생성된 답변을 제출 DataFrame에 추가
submit_df['Answer'] = [item['Answer'] for item in results]
submit_df['Answer'] = submit_df['Answer'].fillna('데이콘')

#결과를 csv 파일로 저장
submit_df.to_csv('./baseline_submission.csv', encoding = 'UTF-8-sig', index = False)
from google.colab import files
files.download('./baseline_submission.csv')

In [20]:
from google.colab import files
files.download('./baseline_submission.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## F1 score

In [11]:
def calculate_f1_score(true_sentence, predicted_sentence, sum_mode=True):

    #공백 제거
    true_sentence = ''.join(true_sentence.split())
    predicted_sentence = ''.join(predicted_sentence.split())

    true_counter = Counter(true_sentence)
    predicted_counter = Counter(predicted_sentence)

    #문자가 등장한 개수도 고려
    if sum_mode:
        true_positive = sum((true_counter & predicted_counter).values())
        predicted_positive = sum(predicted_counter.values())
        actual_positive = sum(true_counter.values())

    #문자 자체가 있는 것에 focus를 맞춤
    else:
        true_positive = len((true_counter & predicted_counter).values())
        predicted_positive = len(predicted_counter.values())
        actual_positive = len(true_counter.values())

    #f1 score 계산
    precision = true_positive / predicted_positive if predicted_positive > 0 else 0
    recall = true_positive / actual_positive if actual_positive > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    return precision, recall, f1_score

def calculate_average_f1_score(true_sentences, predicted_sentences):

    total_precision = 0
    total_recall = 0
    total_f1_score = 0

    for true_sentence, predicted_sentence in zip(true_sentences, predicted_sentences):
        precision, recall, f1_score = calculate_f1_score(true_sentence, predicted_sentence)
        total_precision += precision
        total_recall += recall
        total_f1_score += f1_score

    avg_precision = total_precision / len(true_sentences)
    avg_recall = total_recall / len(true_sentences)
    avg_f1_score = total_f1_score / len(true_sentences)

    return {
        'average_precision': avg_precision,
        'average_recall': avg_recall,
        'average_f1_score': avg_f1_score
    }

# result = calculate_average_f1_score(gt, pred)
# print(result)
# {'average_precision': 0.6231249733675203, 'average_recall': 0.7143688383667051, 'average_f1_score': 0.6224663821869858}

# 모델 학습
1. Retriever 학습(Retrieval model fine-tuning)
2. Generator 학습(LLM fine-tuning)
  - SFT
  - LoRA + 4bit 로 메모리 최적화

## Retreiver 학습
- 학습 데이터 구축(train.csv + chunk 로 query-positive pair 구성)
- E5 기반 학습 (Hugging Face, e5 fine-tuning)
- fine-tuned 모델을 HuggingFaceEmbeddings로 불러오기
- FAISS.from_documents()에 적용해 retriever 교체

In [26]:
#train.csv chunk 생성
base_directory = './data'
df = pd.read_csv('./data/train.csv')


train_chunks = []

for path in df['Source_path'].unique():
    full_path = os.path.join(base_directory, path.lstrip('./'))
    chunks = process_pdf(full_path)
    train_chunks.extend(chunks)



In [27]:
#chunk 중 Answer와 가장 유사한 문장 찾기
from difflib import SequenceMatcher

train_pairs = []

for _, row in df.iterrows():
    answer = row['Answer']
    question = row['Question']
    source_path = row['Source_path']

    # 해당 문서에서만 chunk 선택
    chunks = process_pdf(os.path.join(base_directory, source_path.lstrip('./')))

    # answer와 가장 유사한 chunk 선택
    best_chunk = max(chunks, key=lambda c: SequenceMatcher(None, c.page_content, answer).ratio())

    train_pairs.append({
        "query": question,
        "positive": best_chunk.page_content
    })


In [28]:
train_pairs[0]

{'query': '2024년 중앙정부 재정체계는 어떻게 구성되어 있나요?',
 'positive': '5. 총사업비 관리대상사업\n6. 계속비 대상사업\n주요 재정통계\n●\nⅠ.\n2\n01 재정체계\n▸중앙정부 재정체계는 예산(일반･특별회계)과 기금으로 구분되며, 2024년 기준으로 일반회계 1개, 특별회계 \n21개, 기금 68개로 구성\n∙2024년 예산 지출은 일반회계 356.5조원, 21개 특별회계 81.7조원으로 구성\n∙2024년 기금 지출은 49개 사업성기금 81.2조원, 6개 사회보험성기금 107.1조원, 5개 계정성기금 \n30.1조원으로 구성\n[그림 1-1] 재정지출 구조(2024년 예산 총지출 기준)\n주: 괄호 안은 총계 기준 예산액을 의미\n자료: 디지털예산회계시스템\n2024 주요 재정통계 | 2024 Fiscal Statistics\nⅠ. 주요재정통계\nⅡ. 국제통계\n부록\nⅢ. 분야별 재정지출\nⅠ. 주요재정통계\n3\n▸예산은 ｢국가재정법｣에 근거해 정부가 편성하고 국회가 심의･의결로 확정한 재정지출계획을 의미하며, \n일반회계와 특별회계로 구분\n∙기금은 예산과 구분되는 재정수단으로서 재정운영의 신축성을 기할 필요가 있을 때, 정부가 편성하고 \n국회에서 심의･의결한 기금운용계획에 의해 운용\n[표 1-1] 일반회계･특별회계･기금의 비교\n일반회계\n특별회계\n기금\n설치 사유\n∙국가 고유의 일반적 재정 \n활동\n∙특정 사업 운영\n∙특정 세입으로 특정 세출 \n충당\n∙특정 목적을 위해 특정 자금을 운용\n∙일정 자금을 활용하여 특정 사업을 \n안정적으로 운영\n재원 조달 및 \n운용형태\n∙조세수입\n∙무상 급부\n∙일반회계와 기금의 운용형태 \n혼재\n∙출연금, 부담금 등 다양한 수입원\n∙융자사업 등 기금 고유사업 수행\n확정 절차'}

In [37]:
#SentenceTransformer용 학습 데이터 준비
from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader
examples = [InputExample(texts=[p['query'], p['positive']]) for p in train_pairs]
dataloader = DataLoader(examples, shuffle = True, batch_size = 16)

In [38]:
#MultipleNegativeRankingLoss로 학습
import os
os.environ["WANDB_MODE"] = "disabled"  # ✅ 더 안전한 방법

model = SentenceTransformer('intfloat/multilingual-e5-small')  # or e5-base
loss = losses.MultipleNegativesRankingLoss(model)

model.fit(
    train_objectives=[(dataloader, loss)],
    epochs=3,
    warmup_steps=100,
    output_path="./fine_tuned_e5_retriever",
    show_progress_bar = True
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

Step,Training Loss


In [39]:
model.save('fine_tuned_e5_retriever')

In [44]:
#학습된 모델을 retriever로 사용
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="./fine_tuned_e5_retriever")
db = FAISS.from_documents(chunks, embedding=embeddings)
retriever = db.as_retriever()


In [45]:
!zip -r fine_tuned_e5_retriever.zip fine_tuned_e5_retriever
files.download('fine_tuned_e5_retriever.zip')

  adding: fine_tuned_e5_retriever/ (stored 0%)
  adding: fine_tuned_e5_retriever/tokenizer.json (deflated 76%)
  adding: fine_tuned_e5_retriever/sentencepiece.bpe.model (deflated 49%)
  adding: fine_tuned_e5_retriever/README.md (deflated 66%)
  adding: fine_tuned_e5_retriever/special_tokens_map.json (deflated 86%)
  adding: fine_tuned_e5_retriever/tokenizer_config.json (deflated 77%)
  adding: fine_tuned_e5_retriever/1_Pooling/ (stored 0%)
  adding: fine_tuned_e5_retriever/1_Pooling/config.json (deflated 57%)
  adding: fine_tuned_e5_retriever/config_sentence_transformers.json (deflated 34%)
  adding: fine_tuned_e5_retriever/sentence_bert_config.json (deflated 4%)
  adding: fine_tuned_e5_retriever/model.safetensors (deflated 32%)
  adding: fine_tuned_e5_retriever/modules.json (deflated 62%)
  adding: fine_tuned_e5_retriever/config.json (deflated 49%)
  adding: fine_tuned_e5_retriever/2_Normalize/ (stored 0%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Generator 학습
- retriever가 제공한 context와 Answer를 활용해 텍스트 생성 모델 fine-tuning
- retriever를 이용한 context 생성
- huggingface format으로 저장(jsonl)
- generator ahepf fine-tuning
- 학습 후 적용

In [42]:
#데이터 준비
from datasets import Dataset
import pandas as pd

df = pd.read_csv('./data/train.csv')
df = df[['Question', 'Answer']].dropna()

# 프롬프트 템플릿 적용
def apply_prompt(row):
    return {
        'prompt': f"질문: {row['Question']}\n답변:",
        'completion': row['Answer']
    }

processed = df.apply(apply_prompt, axis=1, result_type='expand')
dataset = Dataset.from_pandas(processed)


In [ ]:
!pip install -U peft

In [43]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from datasets import load_dataset, Dataset
import torch
import pandas as pd


In [44]:
#토크나이저, 모델 불러오기
model_id = "beomi/llama-2-ko-7b"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token  # LLaMA 계열은 pad_token이 없어서 지정 필요

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    quantization_config=bnb_config,
    trust_remote_code=True
)
model = prepare_model_for_kbit_training(model)

#  PEFT LoRA 구성
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, lora_config)



Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

In [80]:
#데이터 토크나이즈
def tokenize_fn(example):
    input_text = example["prompt"]
    target_text = example["completion"]

    # Prompt와 Answer를 하나로 묶어 input으로 사용
    input_ids = tokenizer(
        input_text + target_text,
        truncation=True,
        max_length=512,
        padding="max_length"
    )
    return input_ids

tokenized_dataset = dataset.map(tokenize_fn)


Map:   0%|          | 0/496 [00:00<?, ? examples/s]

In [81]:
tokenized_dataset

Dataset({
    features: ['prompt', 'completion', 'input_ids', 'attention_mask'],
    num_rows: 496
})

In [87]:
def add_labels(example):
    input_ids = example["input_ids"]
    prompt_len = len(tokenizer(example["prompt"]).input_ids)  # 프롬프트 길이 계산
    labels = [-100] * prompt_len + input_ids[prompt_len:]     # 프롬프트는 무시하고 completion만 학습

    example["labels"] = labels
    return example

tokenized_dataset = tokenized_dataset.map(add_labels)

Map:   0%|          | 0/496 [00:00<?, ? examples/s]

In [90]:
# 가비지 컬렉션 및 CUDA 메모리 정리
for name in dir():
    if not name.startswith('_'):
        del globals()[name]

import gc
import torch

gc.collect()
torch.cuda.empty_cache()
torch.cuda.ipc_collect()


In [92]:
#Trainer로 SFT
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./generator-sft",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=8,
    num_train_epochs=3,
    logging_steps=100,
    save_strategy="epoch",
    fp16=True,
    report_to="none",
    gradient_checkpointing = True,
    label_names = ['labels']
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

trainer.train()


Step,Training Loss


TrainOutput(global_step=48, training_loss=1.5797017415364583, metrics={'train_runtime': 1449.338, 'train_samples_per_second': 1.027, 'train_steps_per_second': 0.033, 'total_flos': 3.0490657277607936e+16, 'train_loss': 1.5797017415364583, 'epoch': 3.0})

In [16]:
# trainer.save_model("./generator-sft")
# tokenizer.save_pretrained("./generator-sft")
# !zip -r generator-sft.zip generator-sft
# from google.colab import files
# files.download('generator-sft.zip')

## 학습시킨 retriever, generator를 적용해 RAG 추론
- 학습된 Retriever 로드
- 학습된 Generator 로드
- RAG chain 구성, 추론



In [42]:
for k in pdf_databases.keys():
    print(repr(k))

'1-1 2024 주요 재정통계 1권'
'2024 나라살림 예산개요'
'재정통계해설'
'국토교통부_전세임대(융자)'
'고용노동부_청년일자리창출지원'
'고용노동부_내일배움카드(일반)'
'보건복지부_노인일자리 및 사회활동지원'
'중소벤처기업부_창업사업화지원'
'보건복지부_생계급여'
'국토교통부_소규모주택정비사업'
'국토교통부_민간임대(융자)'
'고용노동부_조기재취업수당'
'2024년도 성과계획서(총괄편)'
'「FIS 이슈 & 포커스」 23-3호 《조세지출 연계관리》'
'「FIS 이슈 & 포커스」 22-3호 《재정융자사업》'
'월간 나라재정 2023년 12월호'


In [43]:
import pandas as pd

# 데이터 로드
train_df = pd.read_csv('./data/train.csv')
test_df = pd.read_csv('./data/test.csv')

# 정규화 함수
import unicodedata
def normalize_string(s):
    return unicodedata.normalize('NFC', str(s)).strip()

# 정규화된 경로 추출
train_paths = set(normalize_string(p) for p in train_df['Source_path'].unique())
test_paths = set(normalize_string(p) for p in test_df['Source_path'].unique())

# test에만 있는 경로
only_in_test = test_paths - train_paths

print("🧾 test.csv에만 있고 train.csv에는 없는 Source_path 목록:")
for path in sorted(only_in_test):
    print("-", path)


🧾 test.csv에만 있고 train.csv에는 없는 Source_path 목록:
- ./test_source/「FIS 이슈 & 포커스」 22-4호 《중앙-지방 간 재정조정제도》.pdf
- ./test_source/「FIS 이슈 & 포커스」 23-2호 《핵심재정사업 성과관리》.pdf
- ./test_source/「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》.pdf
- ./test_source/「FIS 이슈&포커스」 22-2호 《재정성과관리제도》.pdf
- ./test_source/국토교통부_행복주택출자.pdf
- ./test_source/보건복지부_노인장기요양보험 사업운영.pdf
- ./test_source/보건복지부_부모급여(영아수당) 지원.pdf
- ./test_source/산업통상자원부_에너지바우처.pdf
- ./test_source/중소벤처기업부_혁신창업사업화자금(융자).pdf


In [ ]:
def process_pdfs_from_dataframe(df, base_directory):
    '''딕셔너리에 pdf명을 키로하여 db retriever 저장'''
    pdf_databases = {}
    unique_paths = df['Source_path'].unique()

    for path in tqdm(unique_paths, desc='Processing PDFs'):
        normalized_path = normalize_path(path)
        full_path = os.path.normpath(os.path.join(base_directory, normalized_path.lstrip('./'))) if not os.path.isabs(normalized_path) else normalized_path

        pdf_title = os.path.splitext(os.path.basename(full_path))[0]
        print(f'Processing {pdf_title}...')

        # PDF → Chunks
        chunks = process_pdf(full_path)

        # ✅ 학습한 임베딩 모델 경로 사용
        model_path = '/content/drive/MyDrive/Colab Notebooks/dacon_playground/fine_tuned_e5_retriever/fine_tuned_e5_retriever'
        db = create_vector_db(chunks, model_path=model_path)

        # Retriever 생성
        retriever = db.as_retriever(search_type='mmr', search_kwargs={'k': 3, 'fetch_k': 8})

        pdf_databases[pdf_title] = {
            'db': db,
            'retriever': retriever
        }

    return pdf_databases

base_directory = './data'
# 데이터 로드
train_df = pd.read_csv('./data/train.csv')
test_df = pd.read_csv('./data/test.csv')

# 두 데이터셋 병합
combined_df = pd.concat([train_df, test_df], ignore_index=True)

# 중복 경로 제거
combined_df = combined_df.drop_duplicates(subset=['Source_path'])

# retriever 재구축
pdf_databases = process_pdfs_from_dataframe(combined_df, base_directory)


In [45]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()
torch.cuda.ipc_collect()


In [ ]:
#SFT된 generator 로드
from langchain.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch
from transformers import BitsAndBytesConfig

def setup_llm_pipeline(model_path='./generator-sft', use_4bit=False):
    """
    SFT된 LLM을 HuggingFacePipeline으로 불러옵니다.
    model_path: SFT된 모델의 경로
    use_4bit: 4bit 양자화 여부 (True일 경우 bnb_config 적용)
    """

    # 토크나이저 로드
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    tokenizer.use_default_system_prompt = False

    # 모델 로드
    if use_4bit:
        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_quant_type='nf4',
            bnb_4bit_compute_dtype=torch.float16
        )
        model = AutoModelForCausalLM.from_pretrained(
            model_path,
            quantization_config=bnb_config,
            device_map='auto',
            trust_remote_code=True
            )
    else:
        model = AutoModelForCausalLM.from_pretrained(
            model_path,
            device_map='auto',
            trust_remote_code=True
        )

    # HuggingFace Pipeline 생성
    text_generation_pipeline = pipeline(
        model=model,
        tokenizer=tokenizer,
        task='text-generation',
        temperature=0.2,
        return_full_text=False,
        max_new_tokens=128,
    )

    return HuggingFacePipeline(pipeline=text_generation_pipeline)
# SFT된 모델을 4bit로 로드 (메모리 절약)
llm = setup_llm_pipeline(model_path='/content/drive/MyDrive/Colab Notebooks/dacon_playground/generator-sft/generator-sft', use_4bit=True)

# 또는, full precision으로 로드
# llm = setup_llm_pipeline(model_path='./generator-sft', use_4bit=False)


In [ ]:
df = pd.read_csv('./data/test.csv')

#결과를 저장할 리스트 초기화
results = []

#DataFrame의 각 행에 대해 처리
for _, row in tqdm(df.iterrows(), total = len(df), desc = 'Answering Questions'):

  #소스 문자열 정규화
  source = normalize_string(row['Source'])
  question = row['Question']


  #정규화된 키로 데이터베이스 검색
  normalized_keys = {normalize_string(k): v for k, v in pdf_databases.items()}
  retriever = normalized_keys[source]['retriever']


  #RAG 체인 구성
  template = """
  다음 정보를 바탕으로 질문에 답하세요:
  {context}

  질문 : {question}
  답변 :
  """
  prompt = PromptTemplate.from_template(template)

  #RAG 체인 정의
  rag_chain = (
      {'context': retriever | format_docs, 'question': RunnablePassthrough()}
      | prompt
      | llm
      | StrOutputParser()

  )

  #답변 추론
  print(f'Question : {question}')
  full_response = rag_chain.invoke(question)
  print(f'Answer: {full_response}\n')

  #결과 저장
  results.append({
      'Source':row['Source'],
      'Source_path': row['Source_path'],
      'Question': question,
      'Answer':full_response
  })

In [48]:
#제출용 샘플 파일 로드
submit_df = pd.read_csv('./data/sample_submission.csv')

#생성된 답변을 제출 DataFrame에 추가
submit_df['Answer'] = [item['Answer'] for item in results]
submit_df['Answer'] = submit_df['Answer'].fillna('데이콘')

#결과를 csv 파일로 저장
submit_df.to_csv('./sft_submission.csv', encoding = 'UTF-8-sig', index = False)
from google.colab import files
files.download('./sft_submission.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# f1_scores = [char_f1(r["prediction"], r["answer"]) for r in results]
# average_f1 = sum(f1_scores) / len(f1_scores)
# print(f"Average Character-level F1 Score: {average_f1:.4f}")